**INTEGRANTES:**

DOUGLAS SILVA DE ALMEIDA

CARLOS DINART SANTOS DE MORAIS JUNIOR

In [ ]:
!pip install biokit

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from biokit.viz import corrplot

import missingno as msno
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
from sklearn.cluster import KMeans
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Data Science/DataMining/dados.csv').drop('Unnamed: 0', axis=1)
df.head(5)

In [ ]:
df.describe()

In [ ]:
df.info()

150 linhas da coluna inadimplente estão como NaN

In [ ]:
df[df.Inadimplente.isna()]

In [ ]:
df[df.Inadimplente==1]

In [ ]:
df.isna().sum()

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.drop('ID do cliente', axis=1, inplace=True)

##Verificando a Correlação

In [ ]:
df.corr()

In [ ]:
matriz_corr = df.corr()
corr_graf = corrplot.Corrplot(matriz_corr)
corr_graf.plot(upper = 'ellipse', fontsize = 'x-large')
fig = plt.gcf()
fig.set_size_inches(13,8)
fig.show()

**Em azul -> Correlação positiva - se um dado aumenta o outro também aumenta.**

**Em vermelho -> Correlação negativa - se um dado aumenta o outro diminui.**

**O Que é possivel especular de acordo com estas correlações:**


- Renda & Anos de emprego
  - quanto mais experiência no emprego maior a renda
- Renda & Idade
  - quanto mais velho maior a renda
- Renda & Educação (levemente) 
  
  - Provavelmente os dados são de um país com uma menor desigualde social, visto que educação tem apenas um leve impacto na renda.

- Renda & Dividas

  - Quem tem dívidas maiores tb recebe mais

- Dívida do Cartão & debt/income ratio
  - quem tem mais dívidas é mais propenso a dever mais que arrecada
- Dívida do Cartão & anos de emprego
  - Quem trabalha a mais tempo contraí mais dívidas

###Análise Exploratória

In [ ]:


def plot_perc(coluna, df):
  numero_total_de_dados = df.shape[0]
  plt.figure(figsize=(20,8))
  g = sns.countplot(x=coluna, data= df,orient='h')
  g.set_ylabel('Distribuição de idades em %', fontsize=17)
  sizes = []

  for p in g.patches:
    height = p.get_height()
    sizes.append(height)
    g.text(p.get_x() + p.get_width()/1.6, height+1,
           '{:1.2f}%'.format(height/numero_total_de_dados*100),
           ha = 'center', va='bottom', fontsize=12)
    g.set_ylim(0,max(sizes)*1.1)
    


In [ ]:
plot_perc('Anos de emprego', df)

**Estariam os desempregados incluídos em 0 anos de emprego?**

In [ ]:
fig = px.histogram(df, x="Inadimplente", color="Anos de emprego", barmode="group", title="<b>INADIMPLENCIA POR ANOS DE EMPREGO<b>")
fig.update_layout(width=1080, height=700, bargap=0.1)
fig.show()

###Distribuição de clientes por idade

In [ ]:
fig = px.histogram(df, x="Idade", barmode="group", title="<b>Distribuição de clientes por idade<b>",)
fig.update_layout(bargap=0.1)
fig.show()

In [ ]:
#fazendo o gráfico com o percentual de cada idade


In [ ]:
plot_perc('Idade', df)

###Percentual de inadimplentes

In [ ]:
df.Inadimplente.value_counts()

In [ ]:
c_labels = ['Não', 'Sim']
fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]])
fig.add_trace(go.Pie(labels=c_labels, values=df['Inadimplente'].value_counts(), name="Inadimplente"))
fig.update_traces(hole=.4, hoverinfo="label+percent+name", textfont_size=16)
fig.update_layout(
    title_text="Porcentagem de Inadimplentes",
    annotations=[dict(text='Inadimplente', font_size=20, showarrow=False)])
fig.show()

##Normalização dos dados

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
colunas = df.columns

In [ ]:
scaler = StandardScaler()
df_std = scaler.fit_transform(df)
data =pd.DataFrame(df_std, columns=colunas)

In [ ]:
data

###Elbow

In [ ]:
def calcular_wcss(data):
  wcss= []
  for k in range(1,9):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X=data)
    #data['Clusters'] = kmeans.labels_
    wcss.append(kmeans.inertia_)

  return wcss

In [ ]:
soma_dos_quadrados = calcular_wcss(data)

In [ ]:
#plotando o gráfico
plt.figure(figsize=(10,5))
plt.plot(range(1,9),soma_dos_quadrados)
plt.xlabel('Número de Clusters')
plt.show()

In [ ]:
import math 
def numero_otimo_clusters(wcss):
  x1, y1 = 2, wcss[0]
  x2,y2 = 20, wcss[len(wcss)-1]

  distancia = []
  for i in range(len(wcss)):
    x0 = i+2
    y0 = wcss[i]
    numerador = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
    denominador = math.sqrt((y2-y1)**2 + (x2-x1)**2)
    distancia.append(numerador/denominador)
    return distancia.index(max(distancia)) +2

In [ ]:
def elbow(data):
  """ INPUT: pd.Series"""
  k = calcular_wcss(data)
  n_cluster = numero_otimo_clusters(k)
  return n_cluster

In [ ]:
elbow(data)

#plotando o cluster

In [ ]:
n_clust = elbow(data)
agrupador = KMeans(n_clusters=n_clust)
agrupador.fit(data)

In [ ]:
labels= agrupador.labels_
data['Clusters'] = labels

In [ ]:
data

In [ ]:
df_alternativo = data.drop(['Educação', 'Inadimplente', 'Clusters'], axis=1)
df_alternativo

In [ ]:

def plot_scatter(x,y):
  fig = go.Figure()
  fig.add_trace(go.Scatter(x = x, y = y,
                            mode = 'markers',
                          text=labels,
                          marker= dict(color=labels.astype(np.float))))

  fig.update_layout(
      title="Cluster com 2 grupos",
      xaxis_title=f"{x.name}",
      yaxis_title=f"{y.name}"
      )
  fig.show()

In [ ]:
plot_scatter(df['Dívida do cartão'], df['Renda'])

In [ ]:
plot_scatter(df['Educação'], df['Renda'])

In [ ]:
plot_scatter(df['Outras dívidas'], df['Razão débito-renda'])

In [ ]:
plot_scatter(df['Renda'], df['Razão débito-renda'])

In [ ]:

plot_scatter(df['Dívida do cartão'], df['Razão débito-renda'])

##Centroide

In [ ]:
data

In [ ]:
df= scaler.inverse_transform(data.drop('Clusters', axis=1))

In [ ]:
df=pd.DataFrame(df, columns=['Idade', 'Educação', 'Anos de emprego', 'Renda',
       'Dívida do cartão', 'Outras dívidas', 'Inadimplente',
       'Razão débito-renda'])
df['Grupos'] = data['Clusters']

In [ ]:
df.groupby('Grupos').mean()

Grupo 0:

- Alta renda
- Maior Educação
- Possuem mais idade
- Mais tempo no trabalho (possivelmente já possuem carreiras estáveis)

   **⬇ CONSEQUENTEMENTE ⬇**

- Contraem mais dívidas
- São mais inadimplentes
- São clientes de alto lucro e maior risco




Grupo 1:

- Mais jovens
- Menos tempo de emprego
- Menos propensos a fazerem dívidas e comprar produtos caros.
- São clientes de baixo lucro, porém, menor risco.

In [ ]:

grupo_0 = df[df['Grupos']==0]
grupo_1=df[df['Grupos']==1]

In [ ]:
plot_perc('Idade', grupo_0)

In [ ]:
plot_perc('Idade', grupo_1)

In [ ]:
plot_perc('Anos de emprego', grupo_1)

In [ ]:
plot_perc('Anos de emprego', grupo_0)